In [1]:
from import_data_encoded_knn import  *
from sequence_distance_matrix import *
from time import ctime


X_try, X_train, X_test, y_train, y_test, len_X_train = import_data_encoded(encoding = 'binary')

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']

non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']

chem = X_try.smiles

In [2]:
print(ctime())
dist_matr = basic_matrix(X_try, categorical, non_categorical, a_ham = 0.0015789473684210526)
print(ctime())

Fri Aug 14 11:26:02 2020
Fri Aug 14 11:33:46 2020


In [3]:
a_pub = 0.016206896551724137
print(ctime())
dist_matr += a_pub * pubchem2d_matrix(X_try)
print(ctime())

Fri Aug 14 11:34:48 2020
Fri Aug 14 11:55:19 2020


In [9]:
dist_matr = pd.DataFrame(dist_matr)

dist_matr_train = dist_matr.iloc[:len_X_train,:len_X_train]
dist_matr_test = dist_matr.iloc[len_X_train:,:len_X_train]

del dist_matr

In [12]:
neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors=1, n_jobs=-2, leaf_size=80)
neigh.fit(dist_matr_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=80, metric='precomputed',
                     metric_params=None, n_jobs=-2, n_neighbors=1, p=2,
                     weights='uniform')

In [13]:
from sklearn.metrics import accuracy_score
y_pred = neigh.predict(dist_matr_test)
accuracy_score(y_test, y_pred)

0.9055476159542679

In [16]:
# indici dei k composti più vicini ( k=1 )
neigh.kneighbors(dist_matr_test, return_distance = False)

array([[ 2210],
       [11094],
       [  968],
       ...,
       [   35],
       [12144],
       [ 5719]], dtype=int64)

In [180]:
X_test.index[0]

210

In [179]:
X_train.iloc[2210].name

214

In [188]:
uns_step = pd.DataFrame(neigh.kneighbors(dist_matr_test, return_distance = False), columns = ['idx_neigh_train'])

In [190]:
# uns_step = pd.DataFrame(uns_step.idx_neigh_train.apply(lambda x: X_train.iloc[x].name).values,
#                         columns = ['idx_train_neigh'])
uns_step

,idx_neigh_train
0,2210
1,11094
2,968
3,349
4,1158
...,...
7692,12683
7693,1021
7694,35
7695,12144


Devo prendere l'indice dell'esperimento test, l'indice e il label dell'esperimento train 

In [192]:
# uns_step = pd.concat([pd.DataFrame(X_test.index, columns = ['idx_test']), uns_step.idx_train_neigh], axis = 1)
uns_step = pd.concat([pd.DataFrame(X_test.index, columns = ['idx_test']), uns_step.idx_neigh_train], axis = 1)
uns_step

In [194]:
uns_step = pd.concat([uns_step, pd.DataFrame(y_train[uns_step.idx_neigh_train], columns = ['label_train'])], axis = 1)
uns_step

,idx_test,idx_neigh_train,label_train
0,210,2210,1
1,6586,11094,1
2,4516,968,1
3,13815,349,1
4,4556,1158,1
...,...,...,...
7692,12131,12683,0
7693,15545,1021,0
7694,17049,35,0
7695,11551,12144,0


In [195]:
uns_step.to_csv('dataset_prova/unsup_step_RASAR.csv')

# K = 3
### Matrice da rifare... però faccio il codice

In [168]:
neigh3 = KNeighborsClassifier(metric = 'precomputed', n_neighbors=3, n_jobs=-2, leaf_size=80)
neigh3.fit(dist_matr_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=80, metric='precomputed',
                     metric_params=None, n_jobs=-2, n_neighbors=3, p=2,
                     weights='uniform')

In [196]:
tmp = pd.DataFrame(neigh3.kneighbors(dist_matr_test, return_distance = False), columns = ['neigh1', 'neigh2', 'neigh3'])
tmp

,neigh1,neigh2,neigh3
0,2210,7761,7782
1,11094,14998,15437
2,968,4279,5919
3,349,10888,4271
4,1158,2371,15317
...,...,...,...
7692,12683,7189,15064
7693,1021,3672,13659
7694,35,6350,11594
7695,12144,656,1183


In [200]:
labels = pd.DataFrame({'label_neigh1': y_train[tmp.neigh1], 'label_neigh2': y_train[tmp.neigh1],
                       'label_neigh3': y_train[tmp.neigh3]})
labels

,label_neigh1,label_neigh2,label_neigh3
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
...,...,...,...
7692,0,0,0
7693,0,0,0
7694,0,0,1
7695,0,0,0


In [223]:
uns_step_k = labels.T.apply(pd.value_counts).T
uns_step_k

,0,1
0,NaN,3.0
1,NaN,3.0
2,NaN,3.0
3,NaN,3.0
4,NaN,3.0
...,...,...
7692,3.0,NaN
7693,3.0,NaN
7694,2.0,1.0
7695,3.0,NaN


In [224]:
uns_step_k = uns_step_k.fillna(0)
uns_step_k

,0,1
0,0.0,3.0
1,0.0,3.0
2,0.0,3.0
3,0.0,3.0
4,0.0,3.0
...,...,...
7692,3.0,0.0
7693,3.0,0.0
7694,2.0,1.0
7695,3.0,0.0


In [228]:
uns_step_k.columns = ['label_0_count', 'label_1_count']
uns_step_k

,label_0_count,label_1_count
0,0.0,3.0
1,0.0,3.0
2,0.0,3.0
3,0.0,3.0
4,0.0,3.0
...,...,...
7692,3.0,0.0
7693,3.0,0.0
7694,2.0,1.0
7695,3.0,0.0


In [229]:
uns_step_k.to_csv('dataset_prova/unsup_step_RASAR_k.csv')

# Similarity

In [1]:
from import_data_encoded_knn import  *
from sequence_distance_matrix import *
from time import ctime


X_try, X_train, X_test, y_train, y_test, len_X_train = import_data_encoded(encoding = 'binary')

y = np.concatenate((y_train,y_test))
zero_exp = X_try[y == 0].drop_duplicates()
len_zero = zero_exp.shape[0]

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']

non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']


# 0 vs 0

In [3]:
dist_matr = basic_matrix(zero_exp, categorical, non_categorical, a_ham = 0.0015789473684210526)

In [4]:
a_pub = 0.016206896551724137
print(ctime())
dist_matr += a_pub * pubchem2d_matrix(zero_exp)
print(ctime())

Wed Aug 19 12:40:40 2020
Wed Aug 19 12:41:51 2020


In [5]:
dist_matr = pd.DataFrame(dist_matr)
# dist_matr = dist_matr / np.max(dist_matr)

In [6]:
b = pd.DataFrame(columns = ['idx_exp', 'label_exp', 'idx_neigh_0', 'dist_neigh_0'])
y_zero = np.repeat(0, len_zero-1)

for i in range(len_zero):
    print(i, end = ' ')
    neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors=1, n_jobs=-2, leaf_size=80)
    neigh.fit(dist_matr.drop(index = i,columns = i), y_zero)
    nbors = neigh.kneighbors(dist_matr.iloc[i].drop(i).values.reshape((1,-1)), return_distance = True)
    
    cod_exp = zero_exp.iloc[i].name
    #     label_exp = 0
    
    if nbors[1][0][0] > i:
        cod_neigh_0 = zero_exp.iloc[nbors[1][0][0]+1].name
    else:
        cod_neigh_0 = zero_exp.iloc[nbors[1][0][0]].name

    dist_neigh_0 = nbors[0][0][0]
    
    b = b.append({'idx_exp': cod_exp, 'label_exp': 0, 'idx_neigh_0': cod_neigh_0, 'dist_neigh_0': dist_neigh_0},
                ignore_index = True)
    

0 1 2 3 4 5 6 

KeyboardInterrupt: 

In [7]:
b

,idx_exp,label_exp,idx_neigh_0,dist_neigh_0
0,5049.0,0.0,5697.0,0.000121
1,4571.0,0.0,12898.0,0.000121
2,14276.0,0.0,14277.0,0.000486
3,19100.0,0.0,19099.0,0.000121
4,8278.0,0.0,8304.0,0.000364
5,11519.0,0.0,11520.0,0.000121


In [11]:
zero_exp.loc[4571]

species                                                            257
conc1_type                                                           1
exposure_type                                                        3
obs_duration_mean                                                   96
atom_number                                                   0.506647
alone_atom_number                                                    0
bonds_number                                                  0.522111
doubleBond                                                           0
tripleBond                                                           0
ring_number                                                          1
Mol                                                           0.654042
MorganDensity                                                 0.333333
LogP                                                            3.6444
class                                                                1
tax_or

In [12]:
zero_exp.loc[12898] 

species                                                            257
conc1_type                                                           1
exposure_type                                                        9
obs_duration_mean                                                   96
atom_number                                                   0.506647
alone_atom_number                                                    0
bonds_number                                                  0.522111
doubleBond                                                           0
tripleBond                                                           0
ring_number                                                          1
Mol                                                           0.654042
MorganDensity                                                 0.333333
LogP                                                            3.6444
class                                                                1
tax_or

# 1 vs 1

In [1]:
from import_data_encoded_knn import  *
from sequence_distance_matrix import *
from time import ctime


X_try, X_train, X_test, y_train, y_test, len_X_train = import_data_encoded(encoding = 'binary')

y = np.concatenate((y_train,y_test))
one_exp = X_try[y == 1].drop_duplicates()
len_one = one_exp.shape[0]

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']

non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']

In [2]:
dist_matr = basic_matrix(one_exp, categorical, non_categorical, a_ham = 0.0015789473684210526)
a_pub = 0.016206896551724137
print(ctime())
dist_matr += a_pub * pubchem2d_matrix(one_exp)
print(ctime())

dist_matr = pd.DataFrame(dist_matr)
# relativizzo
# dist_matr = dist_matr / np.max(dist_matr)

Wed Aug 19 16:37:33 2020
Wed Aug 19 16:40:57 2020


In [7]:
b = pd.DataFrame(columns = ['idx_exp', 'label_exp', 'idx_neigh_1', 'dist_neigh_1'])
y_one = np.repeat(1, len_one-1)

for i in range(len_one):
    print(i, end = ' ')
    neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors=1, n_jobs=-2, leaf_size=80)
    neigh.fit(dist_matr.drop(index = i,columns = i), y_one)
    nbors = neigh.kneighbors(dist_matr.iloc[i].drop(i).values.reshape((1,-1)), return_distance = True)
    
    cod_exp = one_exp.iloc[i].name
    #     label_exp = 0
    
    if nbors[1][0][0] > i:
        cod_neigh_1 = one_exp.iloc[nbors[1][0][0]+1].name
    else:
        cod_neigh_1 = one_exp.iloc[nbors[1][0][0]].name

    dist_neigh_1 = nbors[0][0][0]
    
    b = b.append({'idx_exp': cod_exp, 'label_exp': 1, 'idx_neigh_1': cod_neigh_1, 'dist_neigh_1': dist_neigh_1},
                ignore_index = True)
    

0 1 2 3 4 

KeyboardInterrupt: 

In [8]:
b

,idx_exp,label_exp,idx_neigh_1,dist_neigh_1
0,848.0,1.0,850.0,0.000121
1,20921.0,1.0,20935.0,0.000364
2,17950.0,1.0,22546.0,0.107191
3,7627.0,1.0,7626.0,0.000121


# 0 vs 1 // 1 vs 0

In [2]:
from import_data_encoded_knn import  *
from sequence_distance_matrix import *
from time import ctime

# da cambiare encoding
X_try, _,_, y_train, y_test, len_X_train = import_data_encoded(encoding = 'binary')

y = np.concatenate((y_train,y_test))

# one_exp = X_try[y == 1].drop_duplicates()
# zero_exp = X_try[y == 0].drop_duplicates()

# len_one = one_exp.shape[0]
# len_zero = zero_exp.shape[0]


categorical = ["exposure_type", "conc1_type","species", 'class', 'tax_order', 'family', 'genus']

non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP', 'oh_count','tripleBond',
                  'doubleBond', 'alone_atom_number','obs_duration_mean', 'ring_number']

In [3]:
a_pub = 0.016206896551724137
print(ctime())
dist_matr = basic_matrix(X_try, categorical, non_categorical, a_ham = 0.0015789473684210526)
print(ctime())

Wed Aug 19 17:34:23 2020
Wed Aug 19 17:38:23 2020


In [4]:
print(ctime())
dist_matr += a_pub * pubchem2d_matrix(X_try)
print(ctime())
dist_matr = pd.DataFrame(dist_matr)

Wed Aug 19 17:41:23 2020
Wed Aug 19 17:52:08 2020


In [6]:
# per riga ci sono i non tossici, per colonna ci sono i tossici
a = dist_matr.iloc[y == 0, y == 1]
a

,0,2,5,6,7,8,9,10,11,13,...,23300,23301,23302,23305,23306,23307,23308,23309,23310,23312
1,2.221361,4.572739,4.966293,13.404620,1.967846,10.505752,2.020928,0.275940,0.870471,2.107353,...,2.041045,2.435215,7.310997,5.351085,3.156594,5.847295,3.869555,1.798404,2.741108,1.967846
3,1.526622,3.776130,4.313958,12.574230,1.363889,9.663395,1.424892,1.214558,0.855888,1.591315,...,1.477684,1.771399,6.562008,4.529117,2.463040,5.061487,3.231626,0.980239,2.068687,1.364011
4,2.252137,4.623582,5.007045,13.474184,2.010262,10.572701,2.079922,0.230398,0.907133,2.142125,...,2.091393,2.486012,7.365867,5.420760,3.214918,5.909449,3.915079,1.823168,2.794800,2.010383
12,4.395183,6.751207,7.138237,15.615704,4.168067,12.707496,4.231475,1.966426,3.048941,4.267246,...,4.242582,4.639957,9.513128,7.558305,5.370638,8.055922,6.053924,3.908759,4.948042,4.168067
16,2.378963,4.727426,5.180334,13.583180,2.175779,10.670248,2.253327,0.330796,1.154063,2.321905,...,2.273488,2.642632,7.509187,5.526701,3.374986,6.034166,4.092676,1.869136,2.954220,2.176386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23317,2.986191,5.345169,5.729200,14.192700,2.744117,11.290714,2.802203,0.541869,1.624917,2.859595,...,2.816803,3.214658,8.091109,6.137547,3.940884,6.632054,4.637114,2.534360,3.521740,2.744117
23318,3.179263,5.544233,5.863462,14.395487,2.921614,11.500120,2.979152,0.815769,1.781596,3.009240,...,2.981160,3.393918,8.256124,6.349166,4.117287,6.822258,4.780285,2.784806,3.698317,2.921371
23319,1.575460,3.892751,4.167266,12.726577,1.279887,9.836636,1.332619,1.132105,0.296676,1.333369,...,1.314855,1.737893,6.563560,4.701207,2.450450,5.147601,3.083379,1.432715,2.030150,1.279887
23320,1.385243,3.736835,4.081894,12.572509,1.099838,9.677660,1.159169,1.140088,0.068647,1.232082,...,1.162326,1.572800,6.443844,4.528102,2.296866,4.999499,2.986575,1.150311,1.876567,1.099838


In [18]:
a.iloc[0][a.iloc[0] == a.iloc[0].values.min()].index[0]

Int64Index([8968], dtype='int64')

In [19]:
a.shape

(8566, 14756)

In [38]:
b = pd.DataFrame(columns = ['idx_exp_0', 'idx_neigh_1'])
len_zero = a.shape[0]
len_one = a.shape[1]
y_zero = np.repeat(1, len_zero-1)

for i in range(len_one):
    print(i, end = ' ')
    
    
    cod_exp = X_try[y==0].iloc[i].name
    
    cod_neigh_1 = X_try.iloc[a.iloc[i][a.iloc[i] == a.iloc[i].values.min()].index[0]].name
    
    
    b = b.append({'idx_exp_0': cod_exp, 'idx_neigh_1': cod_neigh_1},
                ignore_index = True)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

KeyboardInterrupt: 

In [70]:
b

,idx_exp_0,idx_neigh_1
0,5049,5666
1,4571,12897
2,14276,14278
3,19100,19089
4,8278,8254
...,...,...
464,19953,19950
465,448,464
466,7535,7545
467,7365,7075


In [5]:
import pandas as pd
db = pd.read_csv('dataset/db_modelli_smiles_pubchem.csv').drop(columns = ['Unnamed: 0', 'test_cas'])
db.exposure_type.value_counts()

S       13314
F        4136
AQUA     2987
R        2841
P          23
E          16
O           2
SP          1
FD          1
IP          1
Name: exposure_type, dtype: int64